In [2]:
import pandas as pd

import pandasql as ps

def pysqldf(q):
    return ps.sqldf(q, globals())

In [80]:
# extraccion de los datos

clientes = pd.read_csv('cliente.csv', sep=';')
facturas = pd.read_csv('factura.csv', sep=';')
despachos = pd.read_csv('despacho.csv', sep=';')
recogidas = pd.read_csv('recogida.csv', sep=';')
lotes = pd.read_csv('lote.csv', sep=';')
fincas = pd.read_csv('finca.csv', sep=';')
cultivos = pd.read_csv('cultivo.csv', sep=';')
precios = pd.read_csv('precio.csv', sep=';')
usuarios = pd.read_csv('usuario.csv', sep=';')

In [ ]:
# 0. realizar un análisis exploratorio de los datos

In [8]:
clientes.head()
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17 non-null     object
 1   nombre  17 non-null     object
dtypes: object(2)
memory usage: 404.0+ bytes


In [10]:
facturas.head()
facturas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      349 non-null    object 
 1   fecha   349 non-null    object 
 2   total   349 non-null    float64
dtypes: float64(1), object(2)
memory usage: 8.3+ KB


In [14]:
despachos.isnull().sum()

id              0
fecha           0
id_cliente      0
id_factura    529
dtype: int64

In [16]:
recogidas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   id           1000000 non-null  object 
 1   fecha        1000000 non-null  object 
 2   id_lote      1000000 non-null  object 
 3   cantidad     1000000 non-null  float64
 4   id_usuario   1000000 non-null  object 
 5   id_despacho  967284 non-null   object 
dtypes: float64(1), object(5)
memory usage: 45.8+ MB


In [17]:
recogidas.isnull().sum()

id                 0
fecha              0
id_lote            0
cantidad           0
id_usuario         0
id_despacho    32716
dtype: int64

In [19]:
lotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          48 non-null     object
 1   nombre      48 non-null     object
 2   id_finca    48 non-null     object
 3   id_cultivo  48 non-null     object
dtypes: object(4)
memory usage: 1.6+ KB


In [21]:
fincas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5 non-null      object
 1   nombre  5 non-null      object
dtypes: object(2)
memory usage: 212.0+ bytes


In [23]:
cultivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10 non-null     object
 1   nombre  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes


In [25]:
precios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          340 non-null    object 
 1   id_cultivo  340 non-null    object 
 2   valor       340 non-null    float64
 3   fecha       340 non-null    object 
dtypes: float64(1), object(3)
memory usage: 10.8+ KB


In [56]:
pysqldf("""
select distinct strftime('%Y-%m', fecha) from despachos
""")

,"strftime('%Y-%m', fecha)"
0,2023-10
1,2021-01
2,2021-02
3,2021-03
4,2021-04
5,2021-05
6,2021-06
7,2021-07
8,2021-08
9,2021-09


In [69]:
# 1. calcular la cantidad despachada por cada finca por mes

# transfomacion #1: calcular el total despachado por finca por despacho
total_despachado_por_finca = pysqldf("""
select
d.id,   
d.fecha,
f.nombre as finca,
sum(r.cantidad) as total_despachado
from despachos d
    join recogidas r
        on r.id_despacho = d.id
    join lotes l
        on r.id_lote = l.id
    join fincas f
        on l.id_finca = f.id
group by f.nombre, d.fecha
""")

# transfomacion #2: calcular la cantidad despachada por finca por mes
resultado = pysqldf("""
        select 
        finca,
        strftime('%Y-%m', fecha) as mes,
        sum(total_despachado) as total_despachado
        from total_despachado_por_finca
        group by finca, strftime('%Y-%m', fecha)
    """)

In [92]:
# 2. calcular la cantidad recogida por cultivo por mes
recogidas_cultivo_finca = pysqldf("""
select
c.nombre as cultivo,
f.nombre as finca,
l.nombre as lote,
r.fecha,
r.cantidad
from recogidas r
    join lotes l
        on r.id_lote = l.id
    join fincas f
        on l.id_finca = f.id
    join cultivos c
        on l.id_cultivo = c.id;
""")

resultado = pysqldf("""
select
cultivo,
strftime('%Y-%m', fecha) as mes,
sum(cantidad) as total                   
from recogidas_cultivo_finca
group by cultivo, strftime('%Y-%m', fecha)
""")

resultado

,cultivo,mes,total
0,algodón,2021-01,1243386.0
1,algodón,2021-02,1127847.0
2,algodón,2021-03,1241682.0
3,algodón,2021-04,1187040.0
4,algodón,2021-05,1306126.0
...,...,...,...
335,trigo,2023-06,523168.0
336,trigo,2023-07,526532.0
337,trigo,2023-08,545839.0
338,trigo,2023-09,510898.0


In [96]:
# 3. calcular la cantidad recogida por cultivo por finca

pysqldf("""
        select
        finca,
        cultivo,
        sum(cantidad) as total
        from recogidas_cultivo_finca
        group by finca, cultivo
        """)



,finca,cultivo,total
0,El Encanto,algodón,5719727.0
1,El Encanto,café,5892012.0
2,El Encanto,caña,5789038.0
3,El Encanto,maíz,11631636.0
4,El Encanto,sorgo,5858943.0
5,El Resguardo,maíz,11597955.0
6,El Resguardo,papa,5854766.0
7,El Resguardo,trigo,5746758.0
8,La Esperanza,algodón,5857048.0
9,La Esperanza,arroz,11765914.0


In [113]:
# 4. calcular un comparativo mes por mes de la cantidad total recogida entre los años 2021, 2022 y 2023

resultado2021 = pysqldf("""
select
strftime('%Y', fecha) as anio,
strftime('%m', fecha) as mes,
sum(cantidad) as total        
from recogidas
where strftime('%Y', fecha) = '2021'
group by strftime('%Y', fecha), strftime('%m', fecha)
""")

resultado2022 = pysqldf("""
select
strftime('%Y', fecha) as anio,
strftime('%m', fecha) as mes,
sum(cantidad) as total        
from recogidas
where strftime('%Y', fecha) = '2022'
group by strftime('%Y', fecha), strftime('%m', fecha)
""")

resultado2023 = pysqldf("""
select
strftime('%Y', fecha) as anio,
strftime('%m', fecha) as mes,
sum(cantidad) as total        
from recogidas
where strftime('%Y', fecha) = '2023'
group by strftime('%Y', fecha), strftime('%m', fecha)
""")

comparativo_mes = pysqldf("""
select 
r21.mes,
r21.total as total2021,
r22.total as total2022,
r23.total as total2023        
from resultado2021 r21
    join resultado2022 r22
        on r21.mes = r22.mes
    left join resultado2023 r23
        on r21.mes = r23.mes
""")

In [115]:
comparativo_mes.to_csv('comparativo_mes.csv', index=False)

In [ ]:
# 5. calcular el ingreso total por finca. El ingreso total se calcula como la cantidad despachada por el precio que tenía ese cultivo en el mes que se hizo la recogida.